## 04 – Modeling & Analysis  
This notebook builds a combined “master dataset” from the storage company database.  
We’ll join customers, payments, units, and visits for reporting and KPI tracking.


In [ ]:
import pandas as pd
import sqlite3
import os

# detecting correct base path (handles iCloud or local Desktop)
base_dir = os.path.expanduser("~/iCloud Drive/Desktop/storage-company-data")
if not os.path.exists(base_dir):
    base_dir = os.path.expanduser("~/Desktop/storage-company-data")

db_path = os.path.join(base_dir, "Data", "storage_company.db")
conn = sqlite3.connect(db_path)

print("Connected to database:", os.path.abspath(db_path))


Connected to database: /Users/boukaskasbrahim/Desktop/storage-company-data/Data/storage_company.db


In [2]:
query = """
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.city,
    c.country,
    c.sign_up_date,
    c.lease_start,
    c.lease_end,
    u.unit_id,
    u.size_sqm,
    u.monthly_fee AS unit_monthly_fee,
    p.amount AS last_payment_amount,
    p.date AS last_payment_date
FROM customers c
LEFT JOIN units u ON c.unit_id = u.unit_id
LEFT JOIN (
    SELECT customer_id, MAX(date) AS date, amount
    FROM payments
    GROUP BY customer_id
) p ON c.customer_id = p.customer_id;
"""

master = pd.read_sql_query(query, conn)
master.head()


,customer_id,first_name,last_name,city,country,sign_up_date,lease_start,lease_end,unit_id,size_sqm,unit_monthly_fee,last_payment_amount,last_payment_date
0,C101451,Sophia,Gonzalez,Amsterdam,Nederland,25-Feb-2019,10-18-2018,04-Jul-2021,#164,4.0,85.06,216.51,2020-05-15
1,C101334,Ava,Taylor,Amstelveen,Nederland,12-12-2020,20230530,active,u1278,12.0,363.58,303.05,28/05/2016
2,C101761,Evelyn,Hernandez,Haarlemmermeer,Nederland,30.08.2017,2022/05/19,29.08.2025,u1234,3.0,70.63,88.95,28/08/2019
3,C101735,Oliver,Williams,Haarlemmermeer,Ned.,23/05/2022,01-03-2018,31-Jul-2025,U1095,10.0,290.99,36.9,2024/04/27
4,C101576,Harper,Thomas,Leiden,NL,01-04-2021,30/09/2018,2026-04-05,219A,15.0,480.43,84.02,2024-11-16
